<a href="https://colab.research.google.com/github/krishna-kenny/nbaWinNeuralNetModel/blob/main/nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.4/279.4 kB 14.5 MB/s eta 0:00:00


In [2]:
import time
import pandas as pd
from nba_api.stats.endpoints import TeamInfoCommon, TeamGameLogs, PlayerGameLogs
from nba_api.stats.static import teams

# Maximum number of retries for each API call
MAX_RETRIES = 3


def fetch_with_retries(func, *args, **kwargs):
    """Attempts a function call up to MAX_RETRIES with exponential backoff."""
    for attempt in range(MAX_RETRIES):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            wait_time = 2**attempt  # Exponential backoff
            print(f"Error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    print(f"Failed after {MAX_RETRIES} attempts.")
    return None


def get_team_info(seasons):
    print("Fetching team information...")
    nba_teams = teams.get_teams()
    team_data = []

    for season in seasons:
        for team in nba_teams:
            team_info = fetch_with_retries(
                TeamInfoCommon,
                team_id=team["id"],
                season_type_nullable="Regular Season",
                timeout=60,
            )
            if team_info:
                df_team = team_info.get_data_frames()[0]
                team_data.append(df_team)
                time.sleep(0.6)  # Delay to avoid API rate limits
            else:
                print(
                    f"Skipping team {team['full_name']} for season {season} after failed attempts."
                )

    if team_data:
        df_teams = pd.concat(team_data, ignore_index=True)
        df_teams.to_csv("nba_team_data.csv", index=False)
    else:
        print("No team data fetched.")


def get_game_logs(seasons):
    print("Fetching game logs...")
    game_log_data = []

    for season in seasons:
        game_logs = fetch_with_retries(
            TeamGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if game_logs:
            df_game_logs = game_logs.get_data_frames()[0]
            game_log_data.append(df_game_logs)
            time.sleep(0.6)
        else:
            print(f"Skipping game logs for season {season} after failed attempts.")

    if game_log_data:
        df_all_game_logs = pd.concat(game_log_data, ignore_index=True)
        df_all_game_logs.to_csv("nba_game_logs.csv", index=False)
    else:
        print("No game log data fetched.")


def get_player_game_logs(seasons):
    print("Fetching player game logs...")
    player_game_log_data = []

    for season in seasons:
        player_game_logs = fetch_with_retries(
            PlayerGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if player_game_logs:
            df_player_game_logs = player_game_logs.get_data_frames()[0]
            player_game_log_data.append(df_player_game_logs)
            time.sleep(0.6)
        else:
            print(
                f"Skipping player game logs for season {season} after failed attempts."
            )

    if player_game_log_data:
        df_all_player_game_logs = pd.concat(player_game_log_data, ignore_index=True)
        df_all_player_game_logs.to_csv("nba_player_game_logs.csv", index=False)
    else:
        print("No player game log data fetched.")


# Define the list of seasons
seasons = ["2023-24", "2024-25"]

# Run functions to save data to CSV files
get_team_info(seasons[-1])
print("Team information data stored.")

get_game_logs(seasons)
print("Game logs data stored.")

get_player_game_logs(seasons[-2:])  # Fetching for the most recent two seasons only
print("Player game logs data stored.")


Fetching team information...
Team information data stored.
Fetching game logs...
Game logs data stored.
Fetching player game logs...
Player game logs data stored.


In [3]:
import pandas as pd
import numpy as np


# Function to preprocess team data
def preprocess_team_data(input_file, output_file):
    """Load, clean, and save team data by removing unnecessary columns."""
    team_data = pd.read_csv(input_file).dropna()
    columns_to_drop = [
        "TEAM_NAME",
        "TEAM_CITY",
        "SEASON_YEAR",
        "TEAM_CODE",
        "TEAM_DIVISION",
        "MIN_YEAR",
        "MAX_YEAR",
    ]
    team_data = team_data.drop(columns=columns_to_drop)
    team_data = team_data.drop_duplicates()
    team_data.to_csv(output_file, index=False)
    print(f"Team data preprocessed and saved. rows: {team_data.shape}")


# Function to preprocess game logs
def preprocess_game_logs(input_file, output_file):
    """Load, clean, and save game log data with specific transformations."""
    game_logs = pd.read_csv(input_file).dropna()

    # Convert 'GAME_DATE' to datetime and sort by date
    game_logs["GAME_DATE"] = pd.to_datetime(game_logs["GAME_DATE"])
    game_logs_sorted = game_logs.sort_values(by="GAME_DATE", ascending=True)

    # Extract 'TEAM1' and 'TEAM2' from 'MATCHUP' column
    game_logs_sorted["TEAM1"] = game_logs_sorted["MATCHUP"].str.split().str[0]
    game_logs_sorted["TEAM2"] = game_logs_sorted["MATCHUP"].str.split().str[2]

    # Drop columns that are not required
    columns_to_drop = [
        "MATCHUP",
        "AVAILABLE_FLAG",
        "TEAM_NAME",
        "TEAM_ABBREVIATION",
        "GAME_ID",
    ]
    game_logs_cleaned = game_logs_sorted.drop(columns=columns_to_drop)

    # Convert 'WL' to binary format: 'W' becomes 1, 'L' becomes 0
    game_logs_cleaned["WL"] = game_logs_cleaned["WL"].apply(
        lambda result: 1 if result == "W" else 0
    )

    # Convert 'SEASON_YEAR' to integer format, using only the starting year
    game_logs_cleaned["SEASON_YEAR"] = game_logs_cleaned["SEASON_YEAR"].apply(
        lambda year: int(year[:4])
    )

    game_logs_cleaned.to_csv(output_file, index=False)
    print(f"Game logs preprocessed and saved. rows: {game_logs_cleaned.shape}")


# Function to preprocess player game logs
def preprocess_player_game_logs(input_file, output_file):
    """Load, clean, and save player game log data with specific transformations."""
    player_game_logs = pd.read_csv(input_file).dropna()

    # Drop unnecessary columns
    columns_to_drop = [
        "PLAYER_NAME",
        "NICKNAME",
        "TEAM_NAME",
        "TEAM_ABBREVIATION",
        "MATCHUP",
        "GAME_ID",
        "MIN_SEC",
    ]
    player_game_logs = player_game_logs.drop(columns=columns_to_drop)
    player_game_logs["WL"] = player_game_logs["WL"].apply(
        lambda x: 1 if x == "W" else 0
    )
    player_game_logs["SEASON_YEAR"] = player_game_logs["SEASON_YEAR"].apply(
        lambda x: x[:4]
    )

    player_game_logs.to_csv(output_file, index=False)
    print(f"Player game logs preprocessed and saved. rows: {player_game_logs.shape}")


# Function to compute weighted averages
def compute_weighted_avg(player_id, df):
    """Compute weighted averages for a given player."""
    player_rows = df[df["PLAYER_ID"] == player_id].copy()

    # Retain TEAM_ID
    team_id = player_rows["TEAM_ID"].iloc[0]

    # Convert GAME_DATE to a timestamp for weighting
    player_rows["GAME_TIMESTAMP"] = pd.to_datetime(player_rows["GAME_DATE"]).apply(
        lambda x: x.timestamp()
    )
    max_timestamp = player_rows["GAME_TIMESTAMP"].max()

    # Calculate weights based on recency
    player_rows["WEIGHTS"] = np.exp(
        (player_rows["GAME_TIMESTAMP"] - max_timestamp) / 1e7
    )
    player_rows["WEIGHTS"] /= player_rows["WEIGHTS"].sum()

    # Compute weighted average for all columns after WL
    weighted_avg = (
        player_rows.iloc[:, df.columns.get_loc("WL") + 1 :]  # Select columns after WL
        .mul(player_rows["WEIGHTS"], axis=0)  # Multiply each column by weights
        .sum()  # Sum the weighted values for each column
    )
    weighted_avg["TEAM_ID"] = team_id  # Include TEAM_ID in the output
    return weighted_avg


# Function to create feature data
def create_feature_data(input_csv, output_csv):
    """Generate feature data where each player is represented by a single row."""
    # Load the dataset
    df = pd.read_csv(input_csv)
    df.fillna(0, inplace=True)

    # Ensure GAME_DATE is parsed correctly
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])

    # Get unique players
    unique_players = df["PLAYER_ID"].unique()

    # Create a new dataframe to store the features
    feature_data = []

    for player_id in unique_players:
        weighted_avg = compute_weighted_avg(player_id, df)
        weighted_avg["PLAYER_ID"] = player_id  # Retain the player ID
        feature_data.append(weighted_avg)

    # Convert the list to a DataFrame
    feature_df = pd.DataFrame(feature_data)

    # Save to CSV
    feature_df.to_csv(output_csv, index=False)
    print(f"Feature data saved to {output_csv}. rows: {feature_df.shape}")


# Function to compute team-level aggregated features
def create_team_features(player_features_file, team_features_output):
    """
    Generate aggregated features for each team by averaging player statistics.
    """
    import pandas as pd

    # Load the player features data
    player_data = pd.read_csv(player_features_file)

    # Compute team features
    team_features = player_data.groupby("TEAM_ID").mean().reset_index()

    # Load mapping of TEAM_ID to TEAM_ABBREVIATION and additional features
    preprocessed_nba_team_data = pd.read_csv(
        "preprocessed_nba_team_data.csv"
    )
    id_to_abbr_map = preprocessed_nba_team_data.set_index("TEAM_ID")[
        "TEAM_ABBREVIATION"
    ].to_dict()

    # Apply a function to map TEAM_ID to ABBR
    team_features["TEAM_ID"] = team_features["TEAM_ID"].map(id_to_abbr_map)

    # Rename the column
    team_features.rename(columns={"TEAM_ID": "TEAM_ABBREVIATION"}, inplace=True)

    # Merge additional team features
    additional_features = preprocessed_nba_team_data.drop(
        columns=["TEAM_ID", "TEAM_CONFERENCE", "TEAM_SLUG"]
    )
    team_features = team_features.merge(
        additional_features,
        left_on="TEAM_ABBREVIATION",
        right_on="TEAM_ABBREVIATION",
        how="left",
    )

    # Save the resulting team features to a CSV file
    team_features.to_csv(team_features_output, index=False)
    print(
        f"Team feature data saved to {team_features_output}. rows: {team_features.shape}"
    )


# File paths for input and output data
team_data_file = "nba_team_data.csv"
team_data_output = "preprocessed_nba_team_data.csv"

game_data_file = "nba_game_logs.csv"
game_data_output = "preprocessed_nba_game_logs.csv"

player_game_logs_file = "nba_player_game_logs.csv"
player_game_logs_output = "preprocessed_nba_player_game_logs.csv"

# Run the preprocessing functions
preprocess_team_data(team_data_file, team_data_output)
preprocess_game_logs(game_data_file, game_data_output)
preprocess_player_game_logs(player_game_logs_file, player_game_logs_output)
create_feature_data(player_game_logs_output, "nba_player_features.csv")

# File paths for player features and team output
player_features_file = "nba_player_features.csv"
team_features_output = "nba_team_features.csv"

# Run the team feature creation function
create_team_features(player_features_file, team_features_output)


Team data preprocessed and saved. rows: (30, 9)
Game logs preprocessed and saved. rows: (3300, 54)
Player game logs preprocessed and saved. rows: (35471, 62)
Feature data saved to nba_player_features.csv. rows: (660, 61)
Team feature data saved to nba_team_features.csv. rows: (30, 66)


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib  # To save and load the scaler
from tensorflow.keras.backend import round as kround


def prepare_dataset(game_logs_file, features_file):
    """
    Prepare dataset for training using team-specific features.

    Args:
        game_logs_file: CSV file containing game logs with TEAM1, TEAM2, and WL columns.
        features_file: CSV file to save aggregated team features.

    Returns:
        X: Feature matrix for training.
        y: Target vector (win/loss).
        feature_columns: List of feature names used in the dataset.
    """
    # Load game logs
    game_logs = pd.read_csv(game_logs_file)

    # Aggregate features by team
    team_features = (
        game_logs.groupby("TEAM1").mean(numeric_only=True).reset_index().rename(columns={"TEAM1": "TEAM"})
    )
    team_features.to_csv(features_file, index=False)

    # Merge aggregated features for TEAM1 and TEAM2
    game_logs = game_logs.merge(
        team_features.add_suffix("_TEAM1"), left_on="TEAM1", right_on="TEAM_TEAM1"
    ).merge(
        team_features.add_suffix("_TEAM2"), left_on="TEAM2", right_on="TEAM_TEAM2"
    )

    # Drop unnecessary columns
    columns_to_drop = ["TEAM_TEAM1", "TEAM_TEAM2"]
    game_logs.drop(columns=[col for col in columns_to_drop if col in game_logs.columns], inplace=True)

    # Create feature differences and ratios
    numeric_columns = [col for col in game_logs.columns if col.endswith("_TEAM1")]
    for col in numeric_columns:
        base_col = col.replace("_TEAM1", "")
        game_logs[f"{base_col}_DIFF"] = game_logs[f"{base_col}_TEAM1"] - game_logs[f"{base_col}_TEAM2"]
        game_logs[f"{base_col}_RATIO"] = game_logs[f"{base_col}_TEAM1"] / (game_logs[f"{base_col}_TEAM2"] + 1e-5)

    # Handle missing values
    game_logs.fillna(0, inplace=True)

    # Extract features and target
    feature_columns = game_logs.select_dtypes(include=np.number).columns.difference(["WL"])

    # Print the feature names
    print("Feature Columns Used in the Model:")
    print(feature_columns.tolist())

    X = game_logs[feature_columns].to_numpy()
    y = game_logs["WL"].astype(int).to_numpy()

    return X, y, feature_columns


import tensorflow.keras.backend as K

def custom_accuracy(y_true, y_pred):
    """
    Custom accuracy metric to evaluate the model based on given conditions.
    If the prediction is in [0, 0.5) and true label is 0, it's correct.
    If the prediction is in [0.5, 1] and true label is 1, it's correct.
    """
    condition_1 = K.cast(y_pred < 0.5, dtype="float32") * K.cast(y_true == 0, dtype="float32")
    condition_2 = K.cast(y_pred >= 0.5, dtype="float32") * K.cast(y_true == 1, dtype="float32")
    return K.mean(condition_1 + condition_2)


def build_neural_network(input_shape):
    """
    Build a neural network model.

    Args:
        input_shape: Number of input features.

    Returns:
        model: Compiled neural network model.
    """
    model = Sequential([
        Dense(256, activation="relu", input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(128, activation="relu"),
        Dropout(0.3),
        Dense(64, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy", custom_accuracy])

    return model


def train_model(X, y):
    """
    Train a neural network model.

    Args:
        X: Feature matrix for training.
        y: Target vector (win/loss).

    Returns:
        model: Trained neural network model.
        scaler: Fitted scaler for feature normalization.
    """
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Handle class imbalance
    X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_scaled, y)

    # Build and train the model
    model = build_neural_network(X_resampled.shape[1])
    model.fit(X_resampled, y_resampled, epochs=16, batch_size=32, validation_split=0.2)

    return model, scaler


def save_model_and_scaler(model, scaler, model_path="model.h5", scaler_path="scaler.pkl"):
    """
    Save trained model and scaler.
    """
    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    print(f"Model saved to {model_path}, Scaler saved to {scaler_path}")


def save_feature_names(feature_names, feature_names_file="feature_names.pkl"):
    """
    Save feature names for later use during prediction.
    """
    joblib.dump(feature_names, feature_names_file)
    print(f"Feature names saved to {feature_names_file}")


def main():
    game_logs_file = "preprocessed_nba_game_logs.csv"
    features_file = "features.csv"
    model_save_path = "model.h5"
    scaler_save_path = "scaler.pkl"

    # Prepare the dataset
    X, y, feature_columns = prepare_dataset(game_logs_file, features_file)

    if X.size == 0 or y.size == 0:
        print("No data available to train the model.")
        return

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train neural network
    model, scaler = train_model(X_train, y_train)

    # Save the model and scaler
    save_model_and_scaler(model, scaler, model_save_path, scaler_save_path)

    # Save feature names after preparing the dataset
    save_feature_names(feature_columns.tolist())

    # Evaluate the model
    X_test_scaled = scaler.transform(X_test)
    test_loss, test_accuracy, test_custom_accuracy = model.evaluate(X_test_scaled, y_test)
    print(f"Neural Network - Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Custom Accuracy: {test_custom_accuracy}")


if __name__ == "__main__":
    main()


<ipython-input-4-e2d8c9755889>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_logs[f"{base_col}_DIFF"] = game_logs[f"{base_col}_TEAM1"] - game_logs[f"{base_col}_TEAM2"]
<ipython-input-4-e2d8c9755889>:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  game_logs[f"{base_col}_RATIO"] = game_logs[f"{base_col}_TEAM1"] / (game_logs[f"{base_col}_TEAM2"] + 1e-5)
<ipython-input-4-e2d8c9755889>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

Feature Columns Used in the Model:
['AST', 'AST_DIFF', 'AST_RANK', 'AST_RANK_DIFF', 'AST_RANK_RATIO', 'AST_RANK_TEAM1', 'AST_RANK_TEAM2', 'AST_RATIO', 'AST_TEAM1', 'AST_TEAM2', 'BLK', 'BLKA', 'BLKA_DIFF', 'BLKA_RANK', 'BLKA_RANK_DIFF', 'BLKA_RANK_RATIO', 'BLKA_RANK_TEAM1', 'BLKA_RANK_TEAM2', 'BLKA_RATIO', 'BLKA_TEAM1', 'BLKA_TEAM2', 'BLK_DIFF', 'BLK_RANK', 'BLK_RANK_DIFF', 'BLK_RANK_RATIO', 'BLK_RANK_TEAM1', 'BLK_RANK_TEAM2', 'BLK_RATIO', 'BLK_TEAM1', 'BLK_TEAM2', 'DREB', 'DREB_DIFF', 'DREB_RANK', 'DREB_RANK_DIFF', 'DREB_RANK_RATIO', 'DREB_RANK_TEAM1', 'DREB_RANK_TEAM2', 'DREB_RATIO', 'DREB_TEAM1', 'DREB_TEAM2', 'FG3A', 'FG3A_DIFF', 'FG3A_RANK', 'FG3A_RANK_DIFF', 'FG3A_RANK_RATIO', 'FG3A_RANK_TEAM1', 'FG3A_RANK_TEAM2', 'FG3A_RATIO', 'FG3A_TEAM1', 'FG3A_TEAM2', 'FG3M', 'FG3M_DIFF', 'FG3M_RANK', 'FG3M_RANK_DIFF', 'FG3M_RANK_RATIO', 'FG3M_RANK_TEAM1', 'FG3M_RANK_TEAM2', 'FG3M_RATIO', 'FG3M_TEAM1', 'FG3M_TEAM2', 'FG3_PCT', 'FG3_PCT_DIFF', 'FG3_PCT_RANK', 'FG3_PCT_RANK_DIFF', 'FG3_PCT_RANK_

Model saved to model.h5, Scaler saved to scaler.pkl
Feature names saved to feature_names.pkl
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - custom_accuracy: 0.5123 - loss: 5.9632e-04 
Neural Network - Test Loss: 0.0007558608776889741, Test Accuracy: 1.0, Custom Accuracy: 0.5194345116615295


In [5]:
import numpy as np
import pandas as pd
from nba_api.stats.static import teams
import joblib
from tensorflow.keras.models import load_model


def load_feature_names(feature_names_file="feature_names.pkl"):
    """
    Load saved feature names for feature alignment during prediction.
    """
    feature_names = joblib.load(feature_names_file)

    # Print feature names
    print("Feature Names Used in the Model:")
    print(feature_names)

    return feature_names


def get_team_id_by_abbreviation(team_abbreviation):
    """Retrieve the team ID by abbreviation."""
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team["abbreviation"].lower() == team_abbreviation.lower():
            return team["id"]
    raise ValueError(f"Team '{team_abbreviation}' not found! Please enter a valid abbreviation.")


def fetch_team_features(team_abbreviation, features_file, feature_names):
    """
    Retrieve the team-specific features for the given team abbreviation.

    Args:
        team_abbreviation: Abbreviation of the NBA team (e.g., 'LAL').
        features_file: CSV file containing the aggregated team features.
        feature_names: List of features expected by the model.

    Returns:
        numpy array of the team's features.
    """
    team_features = pd.read_csv(features_file)
    team_row = team_features[team_features["TEAM"] == team_abbreviation.upper()]

    if team_row.empty:
        raise ValueError(f"Features for team '{team_abbreviation}' not found in {features_file}.")

    # Ensure only numeric features are returned
    numeric_features = team_row[feature_names].apply(pd.to_numeric, errors='coerce')

    return numeric_features.to_numpy().flatten()


def predict_matchup_win_probability(team1_abbreviation, team2_abbreviation, features_file, model_path="model.h5", scaler_path="scaler.pkl"):
    """
    Predict the win probability for Team 1 in a matchup against Team 2.

    Args:
        team1_abbreviation: Abbreviation of Team 1 (e.g., 'LAL').
        team2_abbreviation: Abbreviation of Team 2 (e.g., 'BOS').
        features_file: CSV file containing aggregated team features.
        model_path: Path to the trained neural network model file.
        scaler_path: Path to the scaler file for feature normalization.
    """
    # Load model, scaler, and feature names
    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    feature_names = load_feature_names()

    # Fetch features for both teams
    team1_features = fetch_team_features(team1_abbreviation, features_file, feature_names)
    team2_features = fetch_team_features(team2_abbreviation, features_file, feature_names)

    # Create matchup feature differences and ratios
    matchup_features = np.concatenate([
        team1_features - team2_features,
        team1_features / (team2_features + 1e-5)  # Avoid division by zero
    ]).reshape(1, -1)

    # Scale the matchup features
    matchup_features_scaled = scaler.transform(matchup_features)

    # Predict win probability for Team 1
    win_probability = model.predict(matchup_features_scaled)[0][0]

    print(f"\nWin Probability for {team1_abbreviation} vs {team2_abbreviation}: {win_probability * 100:.2f}%")


def display_team_data():
    """
    Display available team abbreviations and names for user reference.
    """
    nba_teams = teams.get_teams()
    print("Available NBA Teams:")
    for team in nba_teams:
        print(f"{team['abbreviation']} - {team['full_name']}")


def main():
    """Main function to handle user input and prediction."""
    features_file = "features.csv"  # Path to the features file

    # Display team data before taking user input
    display_team_data()

    team1_abbreviation = input("Enter Team 1 abbreviation (e.g., 'LAL' for Los Angeles Lakers): ").strip()
    team2_abbreviation = input("Enter Team 2 abbreviation (e.g., 'BOS' for Boston Celtics): ").strip()

    try:
        predict_matchup_win_probability(team1_abbreviation, team2_abbreviation, features_file)
    except ValueError as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    main()


Available NBA Teams:
ATL - Atlanta Hawks
BOS - Boston Celtics
CLE - Cleveland Cavaliers
NOP - New Orleans Pelicans
CHI - Chicago Bulls
DAL - Dallas Mavericks
DEN - Denver Nuggets
GSW - Golden State Warriors
HOU - Houston Rockets
LAC - Los Angeles Clippers
LAL - Los Angeles Lakers
MIA - Miami Heat
MIL - Milwaukee Bucks
MIN - Minnesota Timberwolves
BKN - Brooklyn Nets
NYK - New York Knicks
ORL - Orlando Magic
IND - Indiana Pacers
PHI - Philadelphia 76ers
PHX - Phoenix Suns
POR - Portland Trail Blazers
SAC - Sacramento Kings
SAS - San Antonio Spurs
OKC - Oklahoma City Thunder
TOR - Toronto Raptors
UTA - Utah Jazz
MEM - Memphis Grizzlies
WAS - Washington Wizards
DET - Detroit Pistons
CHA - Charlotte Hornets
Enter Team 1 abbreviation (e.g., 'LAL' for Los Angeles Lakers): POR
Enter Team 2 abbreviation (e.g., 'BOS' for Boston Celtics): SAS


Feature Names Used in the Model:
['AST', 'AST_DIFF', 'AST_RANK', 'AST_RANK_DIFF', 'AST_RANK_RATIO', 'AST_RANK_TEAM1', 'AST_RANK_TEAM2', 'AST_RATIO', 'AST_TEAM1', 'AST_TEAM2', 'BLK', 'BLKA', 'BLKA_DIFF', 'BLKA_RANK', 'BLKA_RANK_DIFF', 'BLKA_RANK_RATIO', 'BLKA_RANK_TEAM1', 'BLKA_RANK_TEAM2', 'BLKA_RATIO', 'BLKA_TEAM1', 'BLKA_TEAM2', 'BLK_DIFF', 'BLK_RANK', 'BLK_RANK_DIFF', 'BLK_RANK_RATIO', 'BLK_RANK_TEAM1', 'BLK_RANK_TEAM2', 'BLK_RATIO', 'BLK_TEAM1', 'BLK_TEAM2', 'DREB', 'DREB_DIFF', 'DREB_RANK', 'DREB_RANK_DIFF', 'DREB_RANK_RATIO', 'DREB_RANK_TEAM1', 'DREB_RANK_TEAM2', 'DREB_RATIO', 'DREB_TEAM1', 'DREB_TEAM2', 'FG3A', 'FG3A_DIFF', 'FG3A_RANK', 'FG3A_RANK_DIFF', 'FG3A_RANK_RATIO', 'FG3A_RANK_TEAM1', 'FG3A_RANK_TEAM2', 'FG3A_RATIO', 'FG3A_TEAM1', 'FG3A_TEAM2', 'FG3M', 'FG3M_DIFF', 'FG3M_RANK', 'FG3M_RANK_DIFF', 'FG3M_RANK_RATIO', 'FG3M_RANK_TEAM1', 'FG3M_RANK_TEAM2', 'FG3M_RATIO', 'FG3M_TEAM1', 'FG3M_TEAM2', 'FG3_PCT', 'FG3_PCT_DIFF', 'FG3_PCT_RANK', 'FG3_PCT_RANK_DIFF', 'FG3_PCT_RANK_RA

KeyError: "['AST_DIFF', 'AST_RANK_DIFF', 'AST_RANK_RATIO', 'AST_RANK_TEAM1', 'AST_RANK_TEAM2', 'AST_RATIO', 'AST_TEAM1', 'AST_TEAM2', 'BLKA_DIFF', 'BLKA_RANK_DIFF', 'BLKA_RANK_RATIO', 'BLKA_RANK_TEAM1', 'BLKA_RANK_TEAM2', 'BLKA_RATIO', 'BLKA_TEAM1', 'BLKA_TEAM2', 'BLK_DIFF', 'BLK_RANK_DIFF', 'BLK_RANK_RATIO', 'BLK_RANK_TEAM1', 'BLK_RANK_TEAM2', 'BLK_RATIO', 'BLK_TEAM1', 'BLK_TEAM2', 'DREB_DIFF', 'DREB_RANK_DIFF', 'DREB_RANK_RATIO', 'DREB_RANK_TEAM1', 'DREB_RANK_TEAM2', 'DREB_RATIO', 'DREB_TEAM1', 'DREB_TEAM2', 'FG3A_DIFF', 'FG3A_RANK_DIFF', 'FG3A_RANK_RATIO', 'FG3A_RANK_TEAM1', 'FG3A_RANK_TEAM2', 'FG3A_RATIO', 'FG3A_TEAM1', 'FG3A_TEAM2', 'FG3M_DIFF', 'FG3M_RANK_DIFF', 'FG3M_RANK_RATIO', 'FG3M_RANK_TEAM1', 'FG3M_RANK_TEAM2', 'FG3M_RATIO', 'FG3M_TEAM1', 'FG3M_TEAM2', 'FG3_PCT_DIFF', 'FG3_PCT_RANK_DIFF', 'FG3_PCT_RANK_RATIO', 'FG3_PCT_RANK_TEAM1', 'FG3_PCT_RANK_TEAM2', 'FG3_PCT_RATIO', 'FG3_PCT_TEAM1', 'FG3_PCT_TEAM2', 'FGA_DIFF', 'FGA_RANK_DIFF', 'FGA_RANK_RATIO', 'FGA_RANK_TEAM1', 'FGA_